In [50]:
from bs4 import BeautifulSoup 
from csv import writer

In [51]:
import requests

# Kullanıcı aracısı dizesini istediğiniz gibi değiştirin
custom_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

# İstek başlıklarını ayarlayarak kullanıcı aracısı dizesini değiştirin
headers = {
    "User-Agent": custom_user_agent
}

# İstek gönderme
response = requests.get("https://turbo.az/", headers=headers)

# Yanıtı kontrol etme
if response.status_code == 200:
    print("İstek başarılı!")
else:
    print("İstek başarısız!")


İstek başarılı!


In [52]:
import pandas as pd 
import numpy as np 
import requests 
from bs4 import BeautifulSoup
import threading
import time
from datetime import datetime


In [53]:
print('Başladı')

Başladı


In [54]:
exc_start = time.time()
page_count = 416
urls = list()

In [55]:
for page in range(1, page_count+1):
    url = f'https://turbo.az/autos?page={page}'
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.find_all('a', class_='products-i__link')
    urls.append([a_tag.get('href') for a_tag in items]) 

In [56]:
urls = np.concatenate(urls, axis=None)
urls = np.array(['https://turbo.az' + url for url in urls])
urls = np.unique(urls)

print(f'Url Handling: {time.time() - exc_start}')

Url Handling: 531.133950471878


In [57]:
data_list = list()
errors = list()


In [58]:
def scrape(url):
    
    """
    Scrapes data from a webpage based on the provided URL.

    Args:
        url (str): The URL of the webpage to scrape.
    """
    
    data = pd.DataFrame()
    
    
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    
    owner = soup.find('div', 'product-owner__info')
    
    try:
        data['Satici'] = [owner.find('div', class_='product-owner__info-name').text]
        data['Avtosalon'] = 0
    except AttributeError:
        data['Satici'] = [soup.find('div', class_='product-shop__owner-name').text]
        data['Avtosalon'] = 1
    finally: errors.append(('Satici', url))
        
        
        
    try:
        data['Telefonlar'] = ' && '.join([i.text.strip() for i in soup.find_all('a', class_='product-phones__list-i')])
    except: errors.append(('Telefon', url))
    
    
    try:
        side_box = soup.find('div', class_='product-sidebar__box')
        full_price = side_box.find('div', class_='product-price__i').text.split()
        data['Qiymet'], data['Valyuta'] = [int(''.join(full_price[:len(full_price) - 1]))], [full_price[-1]]
    except: errors.append(('Qiymet', url))
    
    
    
    try:
        props_container = soup.find('div', class_='product-properties')
        props = props_container.find_all('div', class_='product-properties__i')
        keys = [i.find('label').text.strip() for i in props]
        vals = [i.find('span').text.strip() for i in props]

        
        for key, val in zip(keys, vals):
            data[key] = [val]
    except: errors.append(('Props', url))
        
    
    
    try:
        stats = soup.find('ul', class_='product-statistics').find_all('li')
        for i in stats:
            txt = i.text 
            key = txt.split(':')[0]
            val = txt.split(':')[1]
            data[key] = [val]
    except: errors.append(('Stats', url))
    
    try:
        extra = soup.find('ul', class_='product-extras').find_all('li')
        data['Extra'] = [', '.join([i.text.strip() for i in extra])]
    except: errors.append(('Extra', url))
    
    try:
        data['Etrafli'] = ' '.join([i.text.strip().replace('\n', ' ') for i in soup.find('div',
                                            class_='product-description__content js-description-content').find_all('p')])
    except: errors.append(('Etrafli', url))
    
    data['Url'] = [url]
    
    data_list.append(data)

In [59]:
url_count = len(urls)
thread_count = 35

In [60]:
arr = [*range(0, url_count, thread_count)]
length = [*range(url_count)]
i = 0

In [61]:
while True:
    all_threads = []
    try:
        for e in length[arr[i]:arr[i + 1]]:
            thread = threading.Thread(target=scrape, args=(urls[e],))
            all_threads.append(thread)

        for thread in all_threads:
            thread.start()

        for thread in all_threads:
            thread.join()

        i += 1
    except:
        for e in length[arr[i]:]:
            thread = threading.Thread(target=scrape, args=(urls[e],))
            all_threads.append(thread)

        for thread in all_threads:
            thread.start()

        for thread in all_threads:
            thread.join()
        break  

Exception in thread Thread-3373:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'text'
Exception in thread Thread-4206:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above except

Exception in thread Thread-9989:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'text'
Exception in thread Thread-10700:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above excep

Exception in thread     self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
Thread-13206:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self.run()
  File

    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeErrorAttributeError: 'NoneType' object has no attribute 'text'
: 'NoneType' object has no attribute 'text'
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
AttributeError: 'NoneType' object has no attribute 'text'
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'text'
Exception in thread Exception in thread Thread-13222:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
Thread-13220:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
Exception in thread Thread-13227:
Traceback (most rec

        self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\

...

        self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
Exception in thread Thread-13668:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
Exception in thread Thread-13667:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'text'
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self._target(*self._args, **self._kwargs)
  File "C:\Users

AttributeErrorAttributeError: 'NoneType' object has no attribute 'text'
: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'text'
Exception in thread Thread-13673:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
Exception in thread Thread-13674:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
Exception in thread Thread-13678:


Exception in thread Exception in thread Thread-13696:
Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 20, in scrape
AttributeError: 'NoneType' object has no attribute 'text'
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
Thread-13698AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
AttributeError: 'NoneType' object has no attribute 'find'

Du

        self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
AttributeError: 'NoneType' object has no attribute 'text'
self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
        self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Pytho

    self._target(*self._args, **self._kwargs)
  File "C:\Users\Edish\AppData\Local\Temp\ipykernel_14960\2607790136.py", line 23, in scrape
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Edish\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
AttributeError: 'NoneType' object has no attribute 'text'
AttributeError: 'NoneType' object has no attribute 'find'

During handling of the above exception, another exception occurred:

Traceback (mos

In [62]:
main_data = pd.concat(data_list, ignore_index=True)
main_data.to_csv(f'{datetime.now().strftime("Turbo.az %b-%d")}.csv',
                 encoding='utf_8_sig',
                 index=False)


In [63]:
print(*errors, end='\n')
print(f'Execution time: {time.time() - exc_start}')

('Satici', 'https://turbo.az/autos/3906410-daewoo-lanos') ('Satici', 'https://turbo.az/autos/5913048-ford-transit') ('Satici', 'https://turbo.az/autos/4588049-mercedes-c-220') ('Satici', 'https://turbo.az/autos/4866885-nissan-xterra') ('Satici', 'https://turbo.az/autos/6094351-jac-t8') ('Satici', 'https://turbo.az/autos/4166390-lexus-rx-300') ('Satici', 'https://turbo.az/autos/6126245-mercedes-e-220') ('Satici', 'https://turbo.az/autos/5252936-maserati-ghibli') ('Satici', 'https://turbo.az/autos/2450721-land-rover-rr-evoque') ('Satici', 'https://turbo.az/autos/5859451-bmw-525') ('Satici', 'https://turbo.az/autos/5756003-gaz-a21r22-03') ('Satici', 'https://turbo.az/autos/5957815-baic-x7') ('Satici', 'https://turbo.az/autos/4166547-lexus-rx-300') ('Satici', 'https://turbo.az/autos/5818620-lexus-es-250') ('Satici', 'https://turbo.az/autos/5931276-faw-tiger-vh') ('Satici', 'https://turbo.az/autos/5817205-hyundai-santa-fe') ('Satici', 'https://turbo.az/autos/5378169-porsche-taycan-4s') ('Sa

In [64]:
main_data

,Satici,Avtosalon,Telefonlar,Qiymet,Valyuta,Şəhər,Marka,Model,Buraxılış ili,Ban növü,...,Yeniləndi,Baxışların sayı,Extra,Etrafli,Url,Sahiblər,Vəziyyəti,Hansı bazar üçün yığılıb,Yerlərin sayı,Qəzalı
0,Elçin,0,(055) 465-88-88,7200,AZN,Bakı,Daewoo,Lanos,2009,Sedan,...,18.09.2023,1323,"Yüngül lehimli disklər, Mərkəzi qapanma, Kondi...",Maşın əla vəziyyətdədir heç bir prab yoxdu mat...,https://turbo.az/autos/3906410-daewoo-lanos,NaN,NaN,NaN,NaN,NaN
1,Jenya,0,(055) 947-96-69,12600,AZN,Xudat,Ford,Transit,1998,Furqon,...,18.09.2023,3904,ABS,Sade mator.sade nasos.qunde surulen mawindi.ma...,https://turbo.az/autos/5913048-ford-transit,NaN,NaN,NaN,NaN,NaN
2,"Avtosalon ""BRABUS and STARTECH Baku""",1,(050) 620-66-66 && (050) 733-85-55 && (012) 43...,46900,USD,Bakı,Nissan,Xterra,2020,Offroader / SUV,...,04.09.2023,56009,"Yüngül lehimli disklər, ABS, Yağış sensoru, Mə...","NEW 2021 model NISSAN X-TERRA""TITANIUM"" 4x4. ...",https://turbo.az/autos/4866885-nissan-xterra,0,NaN,NaN,NaN,NaN
3,"Avtosalon ""039 Cars Sumqayıt""",1,(050) 399-00-39 && (055) 388-10-39 && (070) 38...,12300,AZN,Sumqayıt,Mercedes,C 220,2004,Sedan,...,17.09.2023,69050,"Yüngül lehimli disklər, ABS, Yağış sensoru, Mə...",Mawinda hec bir vuruq yoxdur. Xodovoy ve elekt...,https://turbo.az/autos/4588049-mercedes-c-220,NaN,"Vuruğu yoxdur, rənglənməyib",NaN,NaN,NaN
4,JAC Motors Azərbaycan,1,(050) 268-90-40 && (050) 251-41-54 && (050) 26...,43500,AZN,Bakı,JAC,T8,2022,Pikap,...,15.09.2023,118013,"Yüngül lehimli disklər, ABS, Mərkəzi qapanma, ...",- 4 il və ya 120.000 km zəmanət Mühərrik : 4-s...,https://turbo.az/autos/6094351-jac-t8,NaN,"Vuruğu yoxdur, rənglənməyib",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10791,İlqar,0,(055) 207-31-89,10300,AZN,Bakı,Opel,Astra,2007,Hetçbek,...,19.09.2023,161,"ABS, Yağış sensoru, Mərkəzi qapanma, Kondisioner",Maşın yaxşı veziyetdedi heç bir problemi yoxdu...,https://turbo.az/autos/7749636-opel-astra,NaN,"Vuruğu yoxdur, rənglənməyib",Avropa,5,NaN
10792,Elsen,0,(055) 962-52-05,8800,AZN,Tovuz,LADA (VAZ),2114,2011,Hetçbek,...,19.09.2023,165,"Yüngül lehimli disklər, Mərkəzi qapanma, Yan p...",Bugune hec bir xerc teleb etmir yaxsi veziyyet...,https://turbo.az/autos/7749662-lada-vaz-2114,NaN,"Vuruğu yoxdur, rənglənməyib",Rusiya,NaN,NaN
10793,Elçin,0,(055) 636-73-93,17800,AZN,Bakı,Toyota,Prius,2010,Liftbek,...,19.09.2023,148,"Yüngül lehimli disklər, ABS, Yağış sensoru, Mə...",Maşin ideal vəziyyətdədir. Harda istesez yoxla...,https://turbo.az/autos/7749670-toyota-prius,1,"Vuruğu yoxdur, rənglənməyib",NaN,NaN,NaN
10794,Murad,0,(050) 456-38-18,1300,AZN,Bərdə,Volkswagen,Golf,1994,Hetçbek,...,19.09.2023,188,NaN,Etiyat hisesi kimi de satılır senedleri qaydas...,https://turbo.az/autos/7749684-volkswagen-golf,1,"Vuruğu var, rənglənməyib",Digər,4,Bəli
